In [1]:
import sys; sys.path.append('../3rdparty/ElasticRods/python')
import elastic_rods, elastic_knots
import numpy as np, matplotlib.pyplot as plt, time, io, os

from helpers import *
from parametric_curves import *
import py_newton_optimizer

from linkage_vis import LinkageViewer as Viewer, CenterlineViewer
from tri_mesh_viewer import PointCloudViewer, PointCloudMesh

%load_ext autoreload
%autoreload 2

import parallelism
parallelism.set_max_num_tbb_threads(1)

In [2]:
#only doubles the nodes
def add_n_nodes(n,nodes): 
    nodes = nodes.reshape(-1,3)
    diff = np.concatenate([nodes[:-1] - nodes[1:], [nodes[-1] - nodes[0]]])
    inbetween = nodes- (diff/2)
    result = np.repeat(nodes,2,axis=0)
    result[1::2] = inbetween
    return result

In [3]:
import time
#right now only for 200 nodes (200 *3 = 600)
def move_linear(rods, target):
    steps = 100
    diff = rods.getDoFs()[:600] - target
    DoFs = rods.getDoFs()
    for i in range(int(steps)): #now we move only half way
        DoFs[:600] -= 1/steps *diff 
        rods.setDoFs(DoFs)
        time.sleep(0.1)
        view.update()

In [4]:
# Load the centerline from file...
file = '../data/CubicLatticeKnots/L100/4_1.txt'
#file = '../color_rods(view, rod_list)data/L400-r0.2-UpTo9Crossings/4_1/0010.obj'
#file = '../data/TestKnot.txt'
#file = '../data/4_1-smooth.obj'
rod_radius = 0.2
material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])
centerline = read_nodes_from_file(file)  # supported formats: obj, txt
centerline = add_n_nodes(100, centerline) #double the amount of nodes
pr = define_periodic_rod(centerline, material)
rod_list = elastic_knots.PeriodicRodList([pr])
A_DoFs = []

In [5]:
view = Viewer(rod_list, width=1024, height=800)
view.show()


Renderer(camera=PerspectiveCamera(aspect=1.28, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0),…

In [6]:
def callback(problem, iteration):
    if iteration % 5 == 0:
        A_DoFs.insert(0,rod_list.getDoFs())
        view.update()
        
optimizerOptions = py_newton_optimizer.NewtonOptimizerOptions()
optimizerOptions.niter = 1000
optimizerOptions.gradTol = 1e-8
hessianShift = 1e-4 * compute_min_eigenval_straight_rod(pr)

problemOptions = elastic_knots.ContactProblemOptions()
problemOptions.contactStiffness = 1e3
problemOptions.dHat = 2*rod_radius

fixedVars = []   # all the degrees of freedom can be modified by the optimizer

In [7]:
# Optimize
def optimize():
    report = elastic_knots.compute_equilibrium(
        rod_list, problemOptions, optimizerOptions, 
        fixedVars=fixedVars,
        externalForces=np.zeros(rod_list.numDoF()),
        softConstraints=[],
        callback=callback,
        hessianShift=hessianShift
        )
    view.update()

In [8]:
def go_backwards():
    for dof in A_DoFs:
        rod_list.setDoFs(dof)
        time.sleep(0.1)
        view.update()

In [9]:
optimize()
time.sleep(1)
go_backwards()
time.sleep(1)
file = '../data/CubicLatticeKnots/L200/4_1.txt'
nodes = read_nodes_from_file(file).flatten()
move_linear(rod_list,nodes)
time.sleep(1)
optimize()
A_DoFs=[]

0	572.763	572.794	572.794	1	1
1	350.632	261.166	261.166	1	1
2	252.789	117.324	117.324	1	1
3	202.474	60.3636	60.3636	1	1
4	187.628	47.5909	47.5909	1	1
5	168.269	38.0267	38.0267	1	1
6	145.046	30.9179	30.9179	1	1
7	119.771	43.1552	43.1552	1	1
8	95.9793	27.9511	27.9511	1	1
9	75.6492	25.6457	25.6457	1	1
10	59.6192	24.8538	24.8538	1	1
11	46.7659	18.3302	18.3302	1	1
12	36.2343	22.5963	22.5963	1	1
13	34.6544	5.99918	5.99918	1	1
14	33.6356	3.2482	3.2482	1	1
15	32.1914	2.84041	2.84041	1	1
16	29.9401	3.30418	3.30418	1	1
17	26.7283	4.57328	4.57328	1	1
18	23.0205	12.9249	12.9249	1	1
19	22.2074	2.72862	2.72862	1	1
20	21.3233	1.80725	1.80725	1	1
21	19.913	3.01727	3.01727	1	1
22	17.9954	4.05252	4.05252	1	1
23	15.8666	4.32717	4.32717	1	1
24	13.7472	4.77585	4.77585	0.15625	0
25	12.7469	5.50861	5.50861	0.375	0
26	11.2533	4.87506	4.87506	0.375	0
27	10.1256	3.95439	3.95439	0.1875	0
28	9.59586	3.43881	3.43881	0.03125	0
29	9.52498	5.73407	5.73407	1	1
30	9.19285	1.45048	1.45048	1	1
31	9.04391	0.636882	0.63688

In [10]:
view.update()

In [36]:
#Same Knot: 4_1: L100, L200, L400, L500
#Different L300

In [37]:
#TODO: zwei minima verbinden und minimieren -> entsteht ein Kreis?
# lineare Transformation mit minimieren kombinieren (loop: linear, min)
# transformation to '../data/4_1-smooth.obj' and move linear towards different minmum -> will it go to it?